🤗 Transformadores
Aprendizado de máquina de última geração para PyTorch, TensorFlow e JAX.

🤗 Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração. O uso de modelos pré-treinados pode reduzir os custos de computação, a pegada de carbono e economizar tempo e recursos necessários para treinar um modelo do zero. Esses modelos suportam tarefas comuns em diferentes modalidades, como:

📝 Processamento de linguagem natural: classificação de texto, reconhecimento de entidade nomeada, resposta a perguntas, modelagem de linguagem, resumo, tradução, múltipla escolha e geração de texto.
🖼️ Visão Computacional: classificação de imagens, detecção de objetos e segmentação.
🗣️ Áudio: reconhecimento automático de fala e classificação de áudio.
🐙 Multimodal: resposta a perguntas em tabelas, reconhecimento óptico de caracteres, extração de informações de documentos digitalizados, classificação de vídeos e resposta visual a perguntas.

🤗 Os transformadores suportam interoperabilidade de estrutura entre PyTorch, TensorFlow e JAX. Isto proporciona flexibilidade para utilizar uma estrutura diferente em cada fase da vida de um modelo; treinar um modelo em três linhas de código em uma estrutura e carregá-lo para inferência em outra. Os modelos também podem ser exportados para um formato como ONNX e TorchScript para implantação em ambientes de produção.

In [ ]:

! pip install transformers datasets


# Rapido tour

Comece a trabalhar com 🤗 Transformers! Seja você um desenvolvedor ou um usuário comum, este tour rápido ajudará você a começar e mostrará como usar o [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/ pipelines#transformers.pipeline) para inferência, carregue um modelo pré-treinado e um pré-processador com uma [AutoClass](https://huggingface.co/docs/transformers/main/en/./model_doc/auto) e treine rapidamente um modelo com PyTorch ou TensorFlow.


Antes de começar, certifique-se de ter todas as bibliotecas necessárias instaladas:

```bash
!pip instala conjuntos de dados de transformadores
```

Você também precisará instalar sua estrutura de aprendizado de máquina preferida:

```bash
pip instalar tensorflow
```

## Pipeline

O [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) é a maneira mais fácil e rápida de usar um modelo pré-treinado para inferência. Você pode usar o [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) pronto para uso para muitas tarefas em diferentes modalidades, algumas das quais que são mostrados na tabela abaixo:

<Dica>

Para obter uma lista completa de tarefas disponíveis, confira a [referência da API do pipeline](https://huggingface.co/docs/transformers/main/en/./main_classes/pipelines).

</Tip>

| **Task**                     | **Description**                                                                                              | **Modality**    | **Pipeline identifier**                       |
|------------------------------|--------------------------------------------------------------------------------------------------------------|-----------------|-----------------------------------------------|
| Text classification          | assign a label to a given sequence of text                                                                   | NLP             | pipeline(task=“sentiment-analysis”)           |
| Text generation              | generate text given a prompt                                                                                 | NLP             | pipeline(task=“text-generation”)              |
| Summarization                | generate a summary of a sequence of text or document                                                         | NLP             | pipeline(task=“summarization”)                |
| Image classification         | assign a label to an image                                                                                   | Computer vision | pipeline(task=“image-classification”)         |
| Image segmentation           | assign a label to each individual pixel of an image (supports semantic, panoptic, and instance segmentation) | Computer vision | pipeline(task=“image-segmentation”)           |
| Object detection             | predict the bounding boxes and classes of objects in an image                                                | Computer vision | pipeline(task=“object-detection”)             |
| Audio classification         | assign a label to some audio data                                                                            | Audio           | pipeline(task=“audio-classification”)         |
| Automatic speech recognition | transcribe speech into text                                                                                  | Audio           | pipeline(task=“automatic-speech-recognition”) |
| Visual question answering    | answer a question about the image, given an image and a question                                             | Multimodal      | pipeline(task=“vqa”)                          |
| Document question answering  | answer a question about a document, given an image and a question                                            | Multimodal      | pipeline(task="document-question-answering")  |
| Image captioning             | generate a caption for a given image                                                                         | Multimodal      | pipeline(task="image-to-text")                |

Comece criando uma instância de [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) e especificando uma tarefa para a qual deseja usá-lo. Neste guia, você usará o [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) para análise de sentimento como exemplo:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

O [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) baixa e armazena em cache um [modelo pré-treinado](https://huggingface.co/distilbert) padrão -base-uncased-finetuned-sst-2-english) e tokenizer para análise de sentimento. Agora você pode usar o `classificador` no seu texto de destino:

In [ ]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9998}]

Se você tiver mais de uma entrada, passe suas entradas como uma lista para [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) para retornar uma lista de dicionários:

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309

O [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) também pode iterar em um conjunto de dados inteiro para qualquer tarefa que você desejar. Para este exemplo, vamos escolher o reconhecimento automático de fala como nossa tarefa:

In [ ]:
import torch
from transformers import pipeline

speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

Carregue um conjunto de dados de áudio (consulte 🤗 Conjuntos de dados [Início rápido](https://huggingface.co/docs/datasets/quickstart#audio) para obter mais detalhes) que você gostaria de iterar. Por exemplo, carregue o conjunto de dados [MInDS-14](https://huggingface.co/datasets/PolyAI/minds14):

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

Você precisa ter certeza de que a taxa de amostragem do conjunto de dados corresponde à amostragem
rate [`facebook/wav2vec2-base-960h`](https://huggingface.co/facebook/wav2vec2-base-960h) foi treinado em:

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

Você precisa ter certeza de que a taxa de amostragem do conjunto de dados corresponde à amostragem
rate [`facebook/wav2vec2-base-960h`](https://huggingface.co/facebook/wav2vec2-base-960h) foi treinado em:

In [ ]:
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']